In [1]:
import pandas as pd 
import numpy as np

dm = pd.read_csv("mandalika21032022_hapususer30_cleanLABELING.csv", usecols=["tweet_clean", "label"])
dm.columns = ["tweet_clean", "label"]
dm.head(10)

,tweet_clean,label
0,"['b ada', 'pawang', 'hujan', 'nusantara', 'fak...",positif
1,"['b', 'ajaib', 'dunia', 'terkadang', 'jadi', '...",positif
2,"['b balap', 'indonesia', 'turun', 'ajang', 'mo...",positif
3,"['b', 'cibir', 'terpana', 'dgn', 'sakti', 'paw...",negatif
4,"['b dana', 'triliun', 'gelar', 'motogp', 'hapu...",negatif
5,"['b dari', 'mandalika', 'presiden', 'iriana', ...",positif
6,"['b hujan', 'deras', 'surut', 'semangat', 'rib...",positif
7,"['b lelah', 'ribut', 'orang', 'gak', 'bis', 'u...",negatif
8,"['b mana', 'momen', 'motogp', 'kesan', 'n', 'n...",positif
9,"['b menkumham', 'sambut', 'dukung', 'laksana',...",positif


In [2]:
#checking the count of the dependent variable
dm['label'].value_counts()

positif    15
negatif     9
Name: label, dtype: int64

In [3]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

dm["tekslist"] = dm["tweet_clean"].apply(convert_text_list)

print(dm["tekslist"][23])

print("\ntype : ", type(dm["tekslist"][23]))

['b yang', 'sibuk', 'ngurusin', 'luar', 'isi', 'nya', 'tonton', 'musim', 'sibuk', 'ngurusin', 'banding', 'isi', 'balap', 'xf', 'xf', 'x', 'x', 'motogpmandalika']

type :  <class 'list'>


In [4]:
def calc_TF(document):
    #perhitungan jumlah kata
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    #perhitungan tf
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

dm["TF_dict"] = dm['tekslist'].apply(calc_TF)

dm["TF_dict"].head()

0    {'b ada': 0.05, 'pawang': 0.05, 'hujan': 0.05,...
1    {'b': 0.058823529411764705, 'ajaib': 0.0588235...
2    {'b balap': 0.058823529411764705, 'indonesia':...
3    {'b': 0.0625, 'cibir': 0.0625, 'terpana': 0.06...
4    {'b dana': 0.1111111111111111, 'triliun': 0.11...
Name: TF_dict, dtype: object

In [5]:
#hasil tf
index = 23

print('%20s' % "term", "\t", "TF\n")
for key in dm["TF_dict"][index]:
    print('%20s' % key, "\t", dm["TF_dict"][index][key])

                term 	 TF

              b yang 	 0.05555555555555555
               sibuk 	 0.1111111111111111
            ngurusin 	 0.1111111111111111
                luar 	 0.05555555555555555
                 isi 	 0.1111111111111111
                 nya 	 0.05555555555555555
              tonton 	 0.05555555555555555
               musim 	 0.05555555555555555
             banding 	 0.05555555555555555
               balap 	 0.05555555555555555
                  xf 	 0.1111111111111111
                   x 	 0.1111111111111111
     motogpmandalika 	 0.05555555555555555


In [6]:
def calc_DF(tfDict):
    count_DF = {}
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(dm["TF_dict"])

In [7]:
#menghitung idf

n_document = len(dm)
def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#penyimpanan kamus idf
IDF = calc_IDF(n_document, DF)

In [8]:
#perhitungan TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#penyimpanan variabel TF-IDF 
dm["TF-IDF_dict"] = dm["TF_dict"].apply(calc_TF_IDF)

In [9]:
# memunculkan nilai TF-IDF
index = 23

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in dm["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", dm["TF_dict"][index][key] ,"\t" , dm["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

              b yang 	 0.05555555555555555 	 0.1155245300933242
               sibuk 	 0.1111111111111111 	 0.2310490601866484
            ngurusin 	 0.1111111111111111 	 0.2761007388653334
                luar 	 0.05555555555555555 	 0.1380503694326667
                 isi 	 0.1111111111111111 	 0.2310490601866484
                 nya 	 0.05555555555555555 	 0.09954219273489194
              tonton 	 0.05555555555555555 	 0.0871453287729914
               musim 	 0.05555555555555555 	 0.1380503694326667
             banding 	 0.05555555555555555 	 0.1380503694326667
               balap 	 0.05555555555555555 	 0.1380503694326667
                  xf 	 0.1111111111111111 	 0.2761007388653334
                   x 	 0.1111111111111111 	 0.12206803207423442
     motogpmandalika 	 0.05555555555555555 	 0.022525839339342466


In [10]:
#matrik tf-idf
# pengurutan descending berdasarkan nilai DF  
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# pembuatan list kata dari pengurutan `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

dm["TF_IDF_Vec"] = dm["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("tampil baris pertama matrix TF_IDF_Vec Series\n")
print(dm["TF_IDF_Vec"][0])

print("\nukuran matrix : ", len(dm["TF_IDF_Vec"][0]))

tampil baris pertama matrix TF_IDF_Vec Series

[0.02027325540540822, 0.030655223644320446, 0.043773436867695, 0.0, 0.0, 0.061607184064631615, 0.061607184064631615, 0.0, 0.0, 0.0, 0.07843079589569227, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1039720770839918, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12424533248940002, 0.12424533248940002, 0.12424533248940002, 0.12424533248940002]

ukuran matrix :  50


In [11]:
#menampilkan top 50 term tf-idf

# konversi ke dalam List
TF_IDF_Vec_List = np.array(dm["TF_IDF_Vec"].to_list())
 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
1,n,0.730494
5,pawang,0.670834
3,motogp,0.599733
7,b,0.595698
4,x,0.577092
2,hujan,0.558998
37,moga,0.528112
8,xe,0.510330
9,xa,0.510330
16,sukses,0.478252


In [ ]:
#PEMODELAN MENGGUNAKAN NAIVE BAYES CLASSIFIER

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

vectorizer = TfidfVectorizer (max_features=2500)
model_g = GaussianNB()

In [13]:
v_data = vectorizer.fit_transform(dm['tweet_clean']).toarray()

print (v_data)

[[0.         0.25370555 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.30739214 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.41212152 0.         0.        ]
 [0.         0.         0.         ... 0.19700754 0.         0.        ]]


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(v_data, dm['label'], test_size=0.2, random_state=0)
model_g.fit(X_train,y_train)

GaussianNB()

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_preds = model_g.predict(X_test)

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[2 2]
 [0 1]]
              precision    recall  f1-score   support

     negatif       1.00      0.50      0.67         4
     positif       0.33      1.00      0.50         1

    accuracy                           0.60         5
   macro avg       0.67      0.75      0.58         5
weighted avg       0.87      0.60      0.63         5

nilai akurasinya adalah  0.6


In [16]:
tweet = ''
v_data = vectorizer.transform([tweet]).toarray()
y_preds = model_g.predict(v_data)

#dengan asumsi bahwa 1 merupakan label positif
if y_preds == 1:
    print('Positif')
else:
    print('Negatif')

Negatif


<ipython-input-16-9658b84ca926>:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if y_preds == 1:
